In [ ]:
import random
from art import *

class Settings:
    COLORS = {
        "BLEU": '\033[34m',
        "ROUGE": '\033[31m',
        "JAUNE": '\033[33m',
        "VERT": '\033[32m',
    }
    
    NB_PLUS_4 = 4
    NB_PLUS_2_PER_COLOR = 2
    NB_PLUS_INV_PER_COLOR = 2
    NB_PLUS_STOP_PER_COLOR = 2
    NB_ZERO_PER_COLOR = 1
    NB_OTHER_PER_COLOR = 2
    MAX_VALUES_CARDS = 9
    
    NB_PLAYER = 2
    NB_CARDS_PER_PLAYER = 7
    
class Card:
    def __init__(self, color, value):
        self.color = color
        self.value = value
    
class Player:
    def __init__(self, pseudo):
        self.pseudo = pseudo
        self.main = []
        
    def addCard(self, card):
        self.main.append(card)
    
    def printMain(self):
        i = 1
        for card in self.main:            
            print("\033[97m",str(i),card.color,card.value)
            i+=1
                       
        print("\033[97m","Piocher")
        
class Game:
    def __init__(self, lstPlayer, lstCard):
        #Liste des joueurs
        self.lstPlayer = lstPlayer
        #Liste des cartes de la pioche
        self.lstCard = lstCard
        #Dernière carte jouée
        self.lastCardPlayed = None
        #Si la partie est terminée
        self.finish = False
        #Le joueur gagnant
        self.winner = None
        #Si le prochain joueur est bloqué
        self.nextBlocked = False
        self.sensInverse = False
        #Joueur actuel
        self.currentPlayer = None
        self.initialisation()
        
    #Initialisation de la partie
    def initialisation(self):
        #Mélange des cartes de la pioche
        random.shuffle(self.lstCard)
        #Distribution des cartes à chaque joueur
        for i in range(Settings.NB_CARDS_PER_PLAYER):
            for player in self.lstPlayer:
                player.addCard(self.lstCard[0])  
                self.lstCard.pop(0)
        
        #Retourner la première carte du jeu jusqu'à ce que ça ne soit pas un +4
        self.lastCardPlayed = self.lstCard[0]
        while self.lastCardPlayed.value == "+4":
            self.lstCard.pop(0)
            self.lastCardPlayed = self.lstCard[0]
    
    def printLstCard(self):
        for card in self.lstCard:
            print(card.color, card.value)
            
    def printLstPlayer(self):
        for player in self.lstPlayer:
            print(player.pseudo)
    
    #Vérification si on peut jouer la carte ou pas (si même valeur, couleur ou +4)
    def verifCardValide(self, card):
        if card.value == self.lastCardPlayed.value or card.color == self.lastCardPlayed.color or card.value == "+4":
            return True
        else:
            return False
    
    #Piocher
    def piocher(self, player, nbCards):
        for i in range(nbCards):
            player.addCard(self.lstCard[0])  
            self.lstCard.pop(0)
    
    #Appliquer l'effet de la carte (s'il y en a un)
    def applyEffectCard(self, card, player):
        if card.value == "X":
            self.nextBlocked = True
        if card.value == "inv":
            self.sensInverse = True
        if card.value == "+2":
            self.nextBlocked = True
            self.piocher(player, 2)
        if card.value == "+4":
            self.nextBlocked = True
            print("Liste des couleurs :")
            for color in Settings.COLORS:
                print(Settings.COLORS[color], color)

            color = None
            while color is None:
                colorNumber = input("Choisissez la prochaine couleur : ")
                try:
                    color = Settings.COLORS[colorNumber]
                except:
                    print("Cette couleur n'existe pas !")
            card.color = color
            self.piocher(player, 4)
        self.lastCardPlayed = card
    
    #Regarder si un joueur n'a plus de cartes en main
    def isWinner(self, player):
        if len(player.main) <= 0:
            self.winner = player
            self.finish = True
            print(player.pseudo + " à gagné !")
        return self.finish
    
    #Regarder s'il n'y a plus de cartes dans la pioche
    def noMoreCards(self):
        if len(self.lstCard) <= 0:
            self.finish = True
            print("La partie est terminée, il n'y a plus de cartes ! EGALITEEEEEEEE !")
            
    #Début des tours de jeu
    def start(self):
        while not self.finish:
            i = 0
            for player in self.lstPlayer:
                self.currentPlayer = player
                if not self.nextBlocked:                    
                    print("Au tour de :", player.pseudo)
                    player.printMain()
                    valideCard = False
                    card = None

                    print(self.lastCardPlayed.color, "Dernière carte jouée : " + self.lastCardPlayed.value)
                    while not valideCard:
                        numCard = input("Choisissez le numéro d'une carte à poser : ")
                        print("------------------------------------------------------------------------------------------------------")
                        if numCard == "P":
                            self.piocher(player, 1)
                            valideCard = True
                        else:                                                
                            try:
                                card = player.main[int(numCard)-1]
                                valideCard = self.verifCardValide(card)

                                if not valideCard:
                                    print("Vous ne pouvez pas jouer cette carte !")
                                else:
                                    nextPlayer = None
                                    if i+1 > len(self.lstPlayer)-1:
                                        nextPlayer = self.lstPlayer[0]
                                    else:
                                        nextPlayer = self.lstPlayer[i+1]
                                    self.applyEffectCard(card, nextPlayer)
                                    player.main.pop(int(numCard)-1)
                            except:
                                print("Vous ne pouvez pas jouer cette carte !")                                
                else:
                    print(player.pseudo + " a été bloqué !")
                    self.nextBlocked = False
                if self.isWinner(player):
                    break
                if self.sensInverse:
                    newLstPlayer = []
                    for j in range(len(self.lstPlayer)):
                        if i-1 < 0:
                            i = len(self.lstPlayer)
                        i-=1
                        newLstPlayer.append(self.lstPlayer[i])
                    self.lstPlayer = newLstPlayer
                    self.sensInverse = False
                    break
                i+=1
            self.noMoreCards()
        
#Générer la liste de cartes dans la pioche
def generateListCard():
    lstCard = []
    for i in range(Settings.NB_PLUS_4):
        lstCard.append(Card("\033[97m", "+4"))
    
    for color in Settings.COLORS:
        for i in range(Settings.NB_PLUS_2_PER_COLOR):
            lstCard.append(Card(Settings.COLORS[color], "+2"))
        for i in range(Settings.NB_PLUS_INV_PER_COLOR):
            lstCard.append(Card(Settings.COLORS[color], "inv"))
        for i in range(Settings.NB_PLUS_STOP_PER_COLOR):
            lstCard.append(Card(Settings.COLORS[color], "X"))
        
        for i in range(Settings.NB_ZERO_PER_COLOR):            
            lstCard.append(Card(Settings.COLORS[color], "0"))
            
        for i in range(1, Settings.MAX_VALUES_CARDS+1): 
            for j in range(Settings.NB_OTHER_PER_COLOR):            
                lstCard.append(Card(Settings.COLORS[color], str(i)))
            
    return lstCard
    
#Générer la liste de joueurs
def generateListPlayer():
    lstPlayer = []
    for i in range(1, Settings.NB_PLAYER+1):
        pseudo = input("Pseudo du joueur " + str(i) + " :")
        lstPlayer.append(Player(pseudo))
    print("------------------------------------------------------------------------------------------------------")
            
    return lstPlayer

game = Game(generateListPlayer(), generateListCard())
game.start()

Pseudo du joueur 1 : a
Pseudo du joueur 2 : b


------------------------------------------------------------------------------------------------------
Au tour de : a
 1  7
 2  2
 3  7
 4  2
 5  0
 6  8
 7  +2
 Piocher
 Dernière carte jouée : 7


Choisissez le numéro d'une carte à poser :  1


------------------------------------------------------------------------------------------------------
Au tour de : b
 1  5
 2  +4
 3  X
 4  9
 5  1
 6  3
 7  4
 Piocher
 Dernière carte jouée : 7


Choisissez le numéro d'une carte à poser :  1


...

------------------------------------------------------------------------------------------------------
Au tour de : b
 1  4
 2  9
 3  0
 4  X
 5  5
 6  9
 Piocher
 Dernière carte jouée : 1


Choisissez le numéro d'une carte à poser :  P


------------------------------------------------------------------------------------------------------
Au tour de : a
 1  3
 2  8
 3  9
 4  inv
 Piocher
 Dernière carte jouée : 1


Choisissez le numéro d'une carte à poser :  P


------------------------------------------------------------------------------------------------------
Au tour de : b
 1  4
 2  9
 3  0
 4  X
 5  5
 6  9
 7  6
 Piocher
 Dernière carte jouée : 1
